In [1]:
import os
import numpy as np 
import pandas as pd
import seaborn
import seaborn as sns
import shap # for SHAP value
import random
import warnings
import matplotlib.pyplot as plt
from matplotlib.pylab import rcParams
from bs4 import BeautifulSoup
import re

# The Natural Language Toolkit, or more commonly NLTK, is a suite of libraries and programs for symbolic and 
# statistical natural language processing for English written in the Python programming language.
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()


#TQDM is a progress bar library with good support for nested loops and Jupyter/IPython notebooks.
from tqdm import tqdm


# Use Keras Tensorflow deeplearning library

from tensorflow import set_random_seed

from keras.utils import to_categorical
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.layers import Dense,Dropout,Embedding,LSTM
from keras.callbacks import EarlyStopping
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam
from keras.models import Sequential


from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.ensemble import RandomForestClassifier

from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz

from sklearn.metrics import roc_curve,auc,make_scorer, accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix



import eli5 # for permutation importance
from eli5.sklearn import PermutationImportance

from pdpbox import pdp, info_plots # flor partial plots 

from scipy import interp

from itertools import cycle

from joblib import dump, load
seed_value = 123
os.environ['PYTHONHASHSEED']=str(seed_value)


Using TensorFlow backend.


In [2]:
os.getcwd()
warnings.filterwarnings("ignore", category=UserWarning, module='bs4')

In [3]:
set_random_seed(123)
random.seed(123)

In [4]:
np.random.seed(123)
pd.options.mode.chained_assignment = None  #hide any pandas warnings
%matplotlib inline

In [5]:
train= pd.read_csv("input/train.csv")
test = pd.read_csv("input/test.csv")

In [6]:
# Train dataset: Need only class as "Sentiment" and text as 'Phrase'
train = train.rename(columns={'class':'Sentiment','tweet':'Phrase'})
train = train.drop(['Unnamed: 0', 'count', 'hate_speech', 'offensive_language', 'neither'],axis=1).copy()

In [7]:
# Test dataset: Need only text as 'Phrase'
test = test.rename(columns={'Text':'Phrase'})
test = test.drop(['Date', 'Favorites', 'Retweets', 'Tweet ID'],axis=1).copy()


In [8]:
# Need to Save the split texts before cleaning and tokenizing
# Collect dependent values and convert to ONE-HOT encoding
# Output using to_categorical
target_t = train.Sentiment.values
y_target_t = to_categorical(target_t)

# Save the texts before tokenizing (must use the same random seed)
X_train_t, X_val_t, y_train_t, y_val_t = train_test_split(train,y_target_t,
                                                          test_size=0.1,
                                                          random_state=123,
                                                          stratify=y_target_t)

In [9]:
def clean_sentences(df):
    tweets = []
    
#     for sent in tqdm(df['Phrase']):
    for sent in df['Phrase']:
        # remove non-alphabetic characters
        tweet_text = re.sub("[^a-zA-Z]"," ", str(sent))
        
        #remove html content
        tweet_text = BeautifulSoup(tweet_text).get_text()
        
        # tokenize
        words = word_tokenize(tweet_text.lower())
        
        # lemmatize each word to its lemma
        lemma_words = [lemmatizer.lemmatize(i) for i in words]
        
        tweets.append(lemma_words)
        
    return(tweets)

In [10]:
# cleaned tweets for both train and test set retrieved

train_sentences = clean_sentences(train)

test_sentences = clean_sentences(test)

In [11]:
# Collect dependent values and convert to ONE-HOT encoding
# Output using to_categorical
target = train.Sentiment.values
y_target = to_categorical(target)
num_classes = y_target.shape[1]

In [12]:
# Split into train and validation sets
X_train, X_val, y_train, y_val = train_test_split(train_sentences,
                                                  y_target,
                                                  test_size=0.2,
                                                  random_state=42,
                                                  stratify=y_target)

In [13]:
# Getting the no of unique words and max length of a tweet available in the list of cleaned tweets
# It is needed for initializing tokenizer of keras and subsequent padding

# Build an unordered collection of unique elements.
unique_words = set()
len_max = 0

# for sent in tqdm(X_train):
for sent in X_train:

    unique_words.update(sent)
    
    if(len_max<len(sent)):
        len_max=len(sent)

# length of the list of unique_words gives the number of unique words

print(len(list(unique_words)))
print(len_max)

28701
53


In [14]:
# Actual tokenizer of keras and convert to sequences

tokenizer = Tokenizer(num_words=len(list(unique_words)))
tokenizer.fit_on_texts(list(X_train))

# texts_to_sequences
# ARGUMENTS: list of texts to turn to sequences
# RETURN: list of sequences (one per text input)

X_train = tokenizer.texts_to_sequences(X_train)
X_val = tokenizer.texts_to_sequences(X_val)
X_test = tokenizer.texts_to_sequences(test_sentences)

In [15]:
# Padding is done to equalize the lengths of all input tweets.
# LTSM networks need all inputs to be same length.
# Therefore, tweets lesser than max length will be made equal using extra zeros at end. This is padding.
# Also, you always have to give a three-dimensional array as an input to your LSTM network

X_train = sequence.pad_sequences(X_train, maxlen=len_max)
X_val = sequence.pad_sequences(X_val, maxlen=len_max)
X_test = sequence.pad_sequences(X_test, maxlen=len_max)

In [16]:
# Early stopping to prevent overfitting deep learning neural network models
# This is a method that allows you to specify an arbitrary large number of training epochs.
# This stops training once the model performance stops improving on a hold out validation dataset


early_stopping = EarlyStopping(min_delta = 0.001, mode = 'max', monitor = 'val_acc', patience = 2)
callback = [early_stopping]

In [17]:
# re-run model everytime new parameter changes (must run sequential to initialize epoch)
# Model with just acc as metric

model = Sequential()
model.add(Embedding(len(list(unique_words)),300,input_length=len_max))
model.add(LSTM(128,dropout=0.5,recurrent_dropout=0.5,return_sequences=True))
model.add(Dense(100,activation='relu'))
model.add(LSTM(64,dropout=0.5,recurrent_dropout=0.5,return_sequences=False))
model.add(Dropout(0.5))
model.add(Dense(num_classes,activation='softmax'))
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['acc'])
model.summary()

W0803 11:58:09.811493 4450584000 deprecation_wrapper.py:119] From /anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0803 11:58:09.817187 4450584000 deprecation_wrapper.py:119] From /anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0803 11:58:09.819895 4450584000 deprecation_wrapper.py:119] From /anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0803 11:58:09.915667 4450584000 deprecation_wrapper.py:119] From /anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0803 11:58:09.927006 4450584000 deprecati

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 53, 300)           8610300   
_________________________________________________________________
lstm_1 (LSTM)                (None, 53, 128)           219648    
_________________________________________________________________
dense_1 (Dense)              (None, 53, 100)           12900     
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                42240     
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 195       
Total params: 8,885,283
Trainable params: 8,885,283
Non-trainable params: 0
_________________________________________________________________


In [18]:
# re-run model everytime new parameter changes (must run sequential to initialize epoch)
# Model with no metrics

model_1 = Sequential()
model_1.add(Embedding(len(list(unique_words)),300,input_length=len_max))
model_1.add(LSTM(128,dropout=0.5,recurrent_dropout=0.5,return_sequences=True))
model_1.add(LSTM(64,dropout=0.5,recurrent_dropout=0.5,return_sequences=False))
model_1.add(Dense(100,activation='relu')) #try elu
model_1.add(Dropout(0.5))
model_1.add(Dense(num_classes,activation='softmax'))
model_1.compile(optimizer='adam',
                loss='categorical_crossentropy',
                metrics=['acc'])
model_1.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 53, 300)           8610300   
_________________________________________________________________
lstm_3 (LSTM)                (None, 53, 128)           219648    
_________________________________________________________________
lstm_4 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dense_3 (Dense)              (None, 100)               6500      
_________________________________________________________________
dropout_2 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 3)                 303       
Total params: 8,886,159
Trainable params: 8,886,159
Non-trainable params: 0
_________________________________________________________________


In [19]:
# create pandas df for y_train
# Describe classifier hate (0), offensive (1), neither or neutral (2)

y_df = pd.DataFrame(y_train)
y_df.sum()

0     1144.0
1    15348.0
2     3330.0
dtype: float32

In [20]:
# Sum up the no. of records for each classifier
hate_num = y_df[0].sum()
offensive_num = y_df[1].sum()
neutral_num = y_df[2].sum()

In [21]:
# Do inverse ratio so that hate and neutral has a higher weight
print(f' K Hate:{1 - hate_num / len(y_df)}')
print(f' S Hate:{1 - (hate_num / len(y_df))}')

print(f'K neutral: {1 - neutral_num / len(y_df)}')
print(f'S neutral: {1 - (neutral_num / len(y_df))}')

# Inverse ratio will give offensive a lower weight
print(f'K Offensive: {1 - offensive_num / len(y_df)}')
print(f'S Offensive: {1 - (offensive_num / len(y_df))}')

 K Hate:0.9422863485016648
 S Hate:0.9422863485016648
K neutral: 0.8320048431036222
S neutral: 0.8320048431036222
K Offensive: 0.22570880839471297
S Offensive: 0.22570880839471297


In [22]:
# fit the model adjusting for epochs, batch, and weight

model.fit(
    X_train, y_train, 
#    validation_data=(X_val,y_val),
    epochs=15, #may not run all due to callback
    batch_size=256, #faster with larger batch_size but it's generalizing
    verbose=1,
    callbacks=callback, #stops training once the model stops improving. Prevents overfitting.
    class_weight={0: .94*100, 1: .23, 2: .83*10} #use inverse ratio to set hate with highest weight (somewhat arbitrary)
) 

W0803 11:58:11.739079 4450584000 deprecation.py:323] From /anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/15
 6656/19822 [=========>....................] - ETA: 32s - loss: 5.2495 - acc: 0.0759

KeyboardInterrupt: 

In [24]:
model_1.fit(
    X_train, y_train, 
    validation_data=(X_val,y_val),
    epochs=15, #may not run all due to callback
    batch_size=256, #faster with larger batch_size but it's generalizing
    verbose=1,
    callbacks=callback, #stops training once the model stops improving. Prevents overfitting.
    class_weight={0: .94*100, 1: .23, 2: .83*10} #use inverse ratio to set hate with highest weight (somewhat arbitrary)
) 

Train on 19822 samples, validate on 4956 samples
Epoch 1/15
19822/19822 [==============================] - 49s 2ms/step - loss: 4.8657 - acc: 0.0672 - val_loss: 3.4696 - val_acc: 0.0577
Epoch 2/15
19822/19822 [==============================] - 46s 2ms/step - loss: 3.1487 - acc: 0.1309 - val_loss: 2.7525 - val_acc: 0.1885
Epoch 3/15
19822/19822 [==============================] - 43s 2ms/step - loss: 1.4560 - acc: 0.2087 - val_loss: 2.5317 - val_acc: 0.2030
Epoch 4/15
19822/19822 [==============================] - 48s 2ms/step - loss: 0.9970 - acc: 0.2184 - val_loss: 2.2588 - val_acc: 0.2060
Epoch 5/15
19822/19822 [==============================] - 42s 2ms/step - loss: 0.7023 - acc: 0.2259 - val_loss: 2.0932 - val_acc: 0.2179
Epoch 6/15
19822/19822 [==============================] - 44s 2ms/step - loss: 0.5327 - acc: 0.3662 - val_loss: 2.1471 - val_acc: 0.3535
Epoch 7/15
19822/19822 [==============================] - 42s 2ms/step - loss: 0.4324 - acc: 0.5958 - val_loss: 1.6161 - val_acc:

In [25]:
# Predict validation sentiment!

y_pred = model.predict(X_val)
# print(y_pred)

y_pred_1 = model_1.predict(X_val)

In [26]:
# Keras and Sklearn read arrays differently
# Create function to convert keras array to show only one highest sentiment result per list
def keras_output_sklearn(y):
    
    result = []
    
    for element in y:
        result.append(np.argmax(element))
        

    return result

In [27]:
# Report for model with 'acc' metric
report = classification_report(keras_output_sklearn(y_val), keras_output_sklearn(y_pred))
print(report)

              precision    recall  f1-score   support

           0       0.18      0.68      0.28       286
           1       0.96      0.63      0.76      3838
           2       0.56      0.91      0.70       832

   micro avg       0.68      0.68      0.68      4956
   macro avg       0.57      0.74      0.58      4956
weighted avg       0.85      0.68      0.72      4956



In [28]:
# Report for model with no metrics 
report_1 = classification_report(keras_output_sklearn(y_val), keras_output_sklearn(y_pred_1))
print(report_1)

              precision    recall  f1-score   support

           0       0.18      0.66      0.28       286
           1       0.96      0.66      0.78      3838
           2       0.60      0.92      0.73       832

   micro avg       0.70      0.70      0.70      4956
   macro avg       0.58      0.74      0.60      4956
weighted avg       0.86      0.70      0.74      4956



In [ ]:
# Saving model with 'acc' metric
model.save('../models2/model_acc.h5')

# Saving model with no metrics
model_1.save('../models2/model_no.h5')

In [71]:
# model.history.history

# X_train_t, X_val_t, y_train_t, y_val_t = train_test_split(train,y_target_t,
#                                                           test_size=0.2,
#                                                           random_state=42,
#                                                           stratify=y_target_t)

y_train_t2

array([[0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       ...,
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.]], dtype=float32)

## New pre-processing

In [23]:
train2 = pd.read_csv("input/train.csv")
train2 = train2.rename(columns={'class':'Sentiment','tweet':'Phrase'})
train2 = train2.drop(['Unnamed: 0', 'count', 'hate_speech', 'offensive_language', 'neither'],axis=1).copy()

target_t2 = train2.Sentiment.values
y_target_t2 = to_categorical(target_t2)

# Save the texts before tokenizing (must use the same random seed)
X_train_t2, X_val_t2, y_train_t2, y_val_t2 = train_test_split(train2,y_target_t2,
                                                          test_size=0.2,
                                                          random_state=123,
                                                          stratify=y_target_t)


In [24]:
X_train_t2.head()

,Sentiment,Phrase
16906,1,RT @PluggedInPiff: @ThoughtsOfRandy stop follo...
23020,1,Y'all bitches wanna be so different and y'all ...
17176,1,RT @SaNtAnAdaGrEaT: I never understood that &#...
8163,0,"Brady heard yall say he was washed, now he was..."
23303,0,You fucking fag &#8220;@baethingape: this is e...


In [25]:
# X_train_t2.Phrase

# X_train_t2.Sentiment

In [26]:
tweets = X_train_t2.Phrase

In [27]:
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.stem.porter import *
import string


import nltk
from nltk.tokenize import sent_tokenize, word_tokenize

from nltk.stem import WordNetLemmatizer 
  
nltk.download('punkt')    
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/stephengriggs/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/stephengriggs/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [28]:
stopwords=stopwords = nltk.corpus.stopwords.words("english")

other_exclusions = ["#ff", "ff", "rt"]
stopwords.extend(other_exclusions)

stemmer = PorterStemmer()



## Omari's version
def preprocess(text_string):
    """
    Accepts a text string and replaces:
    1) urls with URLHERE
    2) lots of whitespace with one instance
    3) mentions with MENTIONHERE
    This allows us to get standardized counts of urls and mentions
    Without caring about specific people mentioned
    """
    space_pattern = '\s+'
    giant_url_regex = ('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|'
       '[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    mention_regex = '@[\w\-\:]+' #<<<<<<added the semicolon after the + to remove : at end of Rt's
    emoji_regex = '&#[0-9\;\:]+'    #<<<<<<<<<remove emoji's .ex; &#1214324
    parsed_text = re.sub(space_pattern, ' ', text_string)
    parsed_text = re.sub(giant_url_regex, '', parsed_text)
    parsed_text = re.sub(mention_regex, '', parsed_text)
    parsed_text = re.sub(emoji_regex,'',parsed_text)
    parsed_text = parsed_text.strip(string.punctuation)
    return parsed_text


def tokenize(tweet):
    """Removes punctuation & excess whitespace, sets to lowercase,
    and stems tweets. Returns a list of stemmed tokens."""
    tweet = " ".join(re.split('\s|(?<!\d)[,.]|[,.](?!\d)', tweet.lower())).strip()
    tokens = [stemmer.stem(t) for t in tweet.split()]
    return tokens



def basic_tokenize(tweet):
    """Same as tokenize but without the stemming"""
    tweet = " ".join(re.split("[^a-zA-Z.,!?]*", tweet.lower())).strip()
    return tweet.split()



vectorizer = TfidfVectorizer(
    #vectorizer = sklearn.feature_extraction.text.CountVectorizer(
    tokenizer=tokenize,
    preprocessor=preprocess,
    ngram_range=(1, 3),
    stop_words=stopwords, #We do better when we keep stopwords
    use_idf=True,
    smooth_idf=False,
    norm=None, #Applies l2 norm smoothing
    decode_error='replace',
    max_features=10000,
    min_df=5,
    max_df=0.501
    )

In [29]:
tfidf = vectorizer.fit_transform(tweets).toarray()
vocab = {v:i for i, v in enumerate(vectorizer.get_feature_names())}
idf_vals = vectorizer.idf_
idf_dict = {i:idf_vals[i] for i in vocab.values()} #keys are indices; values are IDF scores
tfidf.shape

(19822, 5838)

In [30]:
tweet_tags = []
for t in tweets:
    tokens = basic_tokenize(preprocess(t))
    tags = nltk.pos_tag(tokens)
    tag_list = [x[1] for x in tags]
    #for i in range(0, len(tokens)):
    tag_str = " ".join(tag_list)
    tweet_tags.append(tag_str)
        #print(tokens[i],tag_list[i])

In [31]:
#We can use the TFIDF vectorizer to get a token matrix for the POS tags
pos_vectorizer = TfidfVectorizer(
    #vectorizer = sklearn.feature_extraction.text.CountVectorizer(
    tokenizer=None,
    lowercase=False,
    preprocessor=None,
    ngram_range=(1, 3),
    stop_words=None, #We do better when we keep stopwords
    use_idf=False,
    smooth_idf=False,
    norm=None, #Applies l2 norm smoothing
    decode_error='replace',
    max_features=5000,
    min_df=5,
    max_df=0.501,
    )

In [32]:
#Construct POS TF matrix and get vocab dict
pos = pos_vectorizer.fit_transform(pd.Series(tweet_tags)).toarray()
pos_vocab = {v:i for i, v in enumerate(pos_vectorizer.get_feature_names())}
pos.shape

(19822, 542)

In [33]:
#Now get other features
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer as VS
from textstat.textstat import *

sentiment_analyzer = VS()

def count_twitter_objs(text_string):
    """
    Accepts a text string and replaces:
    1) urls with URLHERE
    2) lots of whitespace with one instance
    3) mentions with MENTIONHERE
    4) hashtags with HASHTAGHERE

    This allows us to get standardized counts of urls and mentions
    Without caring about specific people mentioned.
    +
    """
    space_pattern = '\s+'
    giant_url_regex = ('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|'
        '[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    mention_regex = '@[\w\-]+'
    hashtag_regex = '#[\w\-]+'
    parsed_text = re.sub(space_pattern, ' ', text_string)
    parsed_text = re.sub(giant_url_regex, 'URLHERE', parsed_text)
    parsed_text = re.sub(mention_regex, 'MENTIONHERE', parsed_text)
    parsed_text = re.sub(hashtag_regex, 'HASHTAGHERE', parsed_text)
    return(parsed_text.count('URLHERE'),parsed_text.count('MENTIONHERE'),parsed_text.count('HASHTAGHERE'))

def other_features(tweet):
    """This function takes a string and returns a list of features.
    These include Sentiment scores, Text and Readability scores,
    as well as Twitter specific features"""
    ##SENTIMENT
    sentiment = sentiment_analyzer.polarity_scores(tweet)
    
    words = preprocess(tweet) #Get text only
    
    syllables = textstat.syllable_count(words) #count syllables in words
    num_chars = sum(len(w) for w in words) #num chars in words
    num_chars_total = len(tweet)
    num_terms = len(tweet.split())
    num_words = len(words.split())
    avg_syl = round(float((syllables+0.001))/float(num_words+0.001),4)
    num_unique_terms = len(set(words.split()))
    
    ###Modified FK grade, where avg words per sentence is just num words/1
    FKRA = round(float(0.39 * float(num_words)/1.0) + float(11.8 * avg_syl) - 15.59,1)
    ##Modified FRE score, where sentence fixed to 1
    FRE = round(206.835 - 1.015*(float(num_words)/1.0) - (84.6*float(avg_syl)),2)
    
    twitter_objs = count_twitter_objs(tweet) #Count #, @, and http://
    retweet = 0
    if "rt" in words:
        retweet = 1
    features = [FKRA,
                FRE,
                syllables,
                avg_syl,
                num_chars,
                num_chars_total,
                num_terms,
                num_words,
                num_unique_terms,
                sentiment['neg'],
                sentiment['pos'],
                sentiment['neu'],
                sentiment['compound'],
                twitter_objs[2],
                twitter_objs[1],
                twitter_objs[0],
                retweet]
    #features = pandas.DataFrame(features)
    return features

def get_feature_array(tweets):
    feats=[]
    for t in tweets:
        feats.append(other_features(t))
    return np.array(feats)

In [34]:
other_features_names = ["FKRA",
                        "FRE",
                        "num_syllables",
                        "avg_syl_per_word",
                        "num_chars",
                        "num_chars_total",
                        "num_terms",
                        "num_words",
                        "num_unique_words",
                        "vader neg",
                        "vader pos",
                        "vader neu",
                        "vader compound",
                        "num_hashtags",
                        "num_mentions",
                        "num_urls",
                        "is_retweet"]

In [35]:
feats = get_feature_array(tweets)
feats.shape

(19822, 17)

In [36]:
M = np.concatenate([tfidf,pos,feats],axis=1)
M.shape

(19822, 6397)

In [37]:
#Finally get a list of variable names
variables = ['']*len(vocab)
for k,v in vocab.items():
    variables[v] = k

pos_variables = ['']*len(pos_vocab)
for k,v in pos_vocab.items():
    pos_variables[v] = k

feature_names = variables+pos_variables+other_features_names

In [38]:
early_stopping = EarlyStopping(min_delta = 0.001, mode = 'max', monitor = 'val_acc', patience = 2)
callback = [early_stopping]

In [39]:
unique_words = set()
len_max = 0

# for sent in tqdm(X_train):
for sent in tweets:

    unique_words.update(sent)
    
    if(len_max<len(sent)):
        len_max=len(sent)

# length of the list of unique_words gives the number of unique words

print(len(list(unique_words)))
print(len_max)

94
754


In [40]:
M.shape[1]

6397

In [50]:
print(len(list(unique_words)))
print(X_train.shape)



94
(19822, 53)


In [41]:
# re-run model everytime new parameter changes (must run sequential to initialize epoch)
# Model with just acc as metric

model_2 = Sequential()
model_2.add(Embedding(len(list(unique_words)),300,input_length=M.shape[1]))
model_2.add(LSTM(128,dropout=0.5,recurrent_dropout=0.5,return_sequences=True))
model_2.add(LSTM(64,dropout=0.5,recurrent_dropout=0.5,return_sequences=False))
model_2.add(Dense(100,activation='relu')) #try elu
model_2.add(Dropout(0.5))
model_2.add(Dense(num_classes,activation='softmax'))
model_2.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['acc'])
model_2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 6397, 300)         28200     
_________________________________________________________________
lstm_5 (LSTM)                (None, 6397, 128)         219648    
_________________________________________________________________
lstm_6 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dense_5 (Dense)              (None, 100)               6500      
_________________________________________________________________
dropout_3 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 3)                 303       
Total params: 304,059
Trainable params: 304,059
Non-trainable params: 0
_________________________________________________________________


In [43]:
# model_2.fit(
#     M, y_train_t2, 
#     #validation_data=(X_val,y_val),
#     epochs=15, #may not run all due to callback
#     batch_size=256, #faster with larger batch_size but it's generalizing
#     verbose=1,
#     callbacks=callback, #stops training once the model stops improving. Prevents overfitting.
# #    class_weight={0: .94*100, 1: .23, 2: .83*10} #use inverse ratio to set hate with highest weight (somewhat arbitrary)
# ) 

In [44]:
y_pred = model.predict(X_val)

In [46]:
y_pred

array([[0.7849359 , 0.02112642, 0.19393778],
       [0.76216084, 0.02982378, 0.20801541],
       [0.7733667 , 0.02328585, 0.20334744],
       ...,
       [0.7530709 , 0.03221582, 0.21471332],
       [0.75436056, 0.0321478 , 0.2134916 ],
       [0.7680189 , 0.02712601, 0.20485508]], dtype=float32)

In [51]:
num_classes

3